
### Spectroscopic Data Analysis  

        Sean Keenan, PhD Physics  
        Quantum Memories Group, Heriot-Watt University, Edinburgh  
        2021  

In [ ]:
%matplotlib widget
import spec_funcs as sf
import fit_funcs as ff
import numpy as np
import matplotlib.pyplot as mp
import os

mp.style.use('signature.mplstyle')


##### Set-up script  

        Select input folder and polarisation  

In [ ]:
# folder containing all the requisite subfolders and data - refrences included
path = r"C:\Users\sk88\Desktop\0222_BB"
# mark energies / wavelengths of interest
sigma_woi = [595.24]
pi_woi = [604.23, 605.36]
woi = sigma_woi + pi_woi
# calibrate wavelength of OSA
calibration = 1.054269203603
# reference power correction
correction = 150/50
# reference names
refs = ("ref", "reference", "R")
# file extension types
exts = ('.csv', '.txt', '.CSV')
exceptions = ("notes", "setup", "ignore", "Thermal OD")
labels = 'wavelength (nm)', 'OD'


##### Import Raw Spectroscopy Data
        Load raw data from csv file (delimiter is not a problem) and sort into groups - polarisations -> reference / spectrum


In [ ]:
folder_list, file_list = sf.dir_interogate(path, exts, exceptions)
path_names = sf.read_files(folder_list, file_list)
ref_names, data_names = sf.search_paths(path_names, refs)
polarisations = sf.find_numbers(ref_names, tail=1)
ref_paths = [os.path.join(path, x) for x in ref_names]
data_paths = [os.path.join(path, x) for x in data_names]
ref_data, ref_metadata = sf.data_extract(ref_paths, polarisations)
data_sets, metadata_sets = sf.data_extract(data_paths, polarisations)
wave_sets, od_sets = sf.OD_calc(ref_data, data_sets, correction=True, c_factor=correction)
shifted_sets = sf.data_shift(wave_sets, calibration)


##### Main calculations

        Correct OD for background
        Create two data sets - corrected and subtracted 

In [ ]:
backgrounds = [[sf.bin_data(od) for od in od_set] for od_set in od_sets]
od_corrected = [[[x - mean_value for x in od] for od, mean_value in zip(od_set, background)] for od_set, background in zip(od_sets, backgrounds)]
od_ground = [od_corrected[i][0] for i in range(len(od_corrected))]
od_subtracted = [[[x-y for x, y in zip(excited, ground)] for excited in od_corrected[i]] for i, ground in enumerate(od_ground)]
minimums = [[min(od_sub) for od_sub in od_sub_set] for od_sub_set in od_subtracted]
od_subtracted =  [[[x - minimum for x in od_sub] for od_sub, minimum in zip(od_sub_set, minimum)] for od_sub_set, minimum in zip(od_subtracted, minimums)]


##### Find Peaks and Energies

        Calculate frequencies, wavenumbers and wavelengths of peaks

In [ ]:
corrected_peaks = sf.peak_find(shifted_sets, od_corrected, prom_tol=None, top_tol=0.1, lims=None)
sub_peaks = sf.peak_find(shifted_sets, od_subtracted, prom_tol=None, top_tol=0.01, lims=None)
od_frequencies = sf.peak_freq(od_corrected, shifted_sets, lims=[490, 510])
od_wavenumbers = [[[sf.converter(frequency, d_type=1, c_type=0) for frequency in frequency_set] for frequency_set in frequency_list] for frequency_list in od_frequencies]
od_wavelengths = [[[sf.converter(frequency, d_type=1, c_type=2) for frequency in frequency_set] for frequency_set in frequency_list] for frequency_list in od_frequencies]
sub_frequencies = sf.peak_freq(sub_peaks, wave_sets, lims=[490, 510])
sub_wavenumbers = [[[sf.converter(frequency, d_type=1, c_type=0) for frequency in frequency_set] for frequency_set in frequency_list] for frequency_list in sub_frequencies]
sub_wavelengths = [[[sf.converter(frequency, d_type=1, c_type=2) for frequency in frequency_set] for frequency_set in frequency_list] for frequency_list in sub_frequencies]

In [ ]:
sf.plotter(shifted_sets, od_subtracted, keys=polarisations, axis_lbls=labels, shifter=0.1, lims=(), woi=(), save=False, data_labels=folder_list)

In [ ]:
sf.plotter(shifted_sets, od_corrected, keys=polarisations, axis_lbls=labels, lims=(570, 625), shifter=0.2, woi=None, save=False, data_labels=folder_list)

In [ ]:
from scipy.signal import find_peaks, argrelmax, argrelmin

low, high = sf.zoom(shifted_sets[0][2], (603, 610))
test_x = np.array(shifted_sets[0][2])
test_y = np.array(od_subtracted[0][2])
grad_y = np.gradient(test_y)
grad_2y = np.gradient(grad_y)
a = argrelmax(test_y)
b = a[0].tolist()
c = [x for x in b if x >= low and x <= high]
fig, ax = mp.subplots()
ax.plot(test_x[low:high], test_y[low:high], 'r')
ax.plot(test_x[c], test_y[c], 'xr')
ax.plot(test_x[low:high], grad_y[low:high], '--b')
ax.plot(test_x[c], grad_y[c], 'xb')
ax.plot(test_x[low:high], grad_2y[low:high], '--g')
ax.plot(test_x[c], grad_2y[c], 'xg')

#fit, err = ff.fitNguass()

In [ ]:
low_128, high_128 = sf.zoom(shifted_sets[0][0], bounds=(605, 620))
low_173, high_173 = sf.zoom(shifted_sets[0][0], bounds=(596, 610))
peak_sub_128 = [[sf.find_OD(y_values=od, peaks=peaks, lims=(low_128, high_128)) for od, peaks in zip(od_set, peak_set)] for od_set, peak_set in zip(od_subtracted, sub_peaks)]
peak_sub_173 = [[sf.find_OD(y_values=od, peaks=peaks, lims=(low_173, high_173)) for od, peaks in zip(od_set, peak_set)] for od_set, peak_set in zip(od_subtracted, sub_peaks)]

In [ ]:
low_128, high_128 = sf.zoom(shifted_sets[0][0], bounds=(603, 607))
low_173, high_173 = sf.zoom(shifted_sets[0][0], bounds=(594, 597))
peak_cor_128 = [[sf.find_OD(y_values=od, peaks=peaks, lims=(low_128, high_128)) for od, peaks in zip(od_set, peak_set)] for od_set, peak_set in zip(od_corrected, corrected_peaks)]
peak_cor_173 = [[sf.find_OD(y_values=od, peaks=peaks, lims=(low_173, high_173)) for od, peaks in zip(od_set, peak_set)] for od_set, peak_set in zip(od_corrected, corrected_peaks)]

In [ ]:
peak_0 = [value for values in peak_cor_128[0] for value in values]
od_frequencies123 = sf.peak_freq(corrected_peaks, shifted_sets, lims=[603,607])
freq_0 = [frequency for frequencies in od_frequencies123[0] for frequency in frequencies]
wavenumber_0 = [sf.converter(freq, 1, 0) for freq in freq_0]
print(wavenumber_0)

temp_0 = [3.2, 10, 20, 30, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280]
fig_1, ax_1 = mp.subplots()
ax_1.set_title('$\sigma$ polarisation at 16529.87 - 16520.30 cm$^{-1}$')
ax_1.set(xlabel='Temperature', ylabel='OD')
ax_1.plot(temp_0, peak_0, 'x')

In [ ]:
peak_2 = [peak_sub_128[0][i][0] for i in range(len(peak_sub_128[0])) if i != 0]
sub_frequencies = sf.peak_freq(sub_peaks, shifted_sets, lims=[607,620])
freq_2 = [sub_frequencies[0][i][0] for i in range(len(sub_frequencies[0])) if i != 0]
wavenumber_2 = [sf.converter(freq, d_type=1, c_type=0) for freq in freq_2]
print(wavenumber_2)

temp_2 = [60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280]
peak_2.pop(0)
peak_2.pop(1)
peak_2.pop(2)
peak_2.pop(3)
fig_1, ax_1 = mp.subplots()
ax_1.set_title('$\sigma$ polarisation at 16472.69 cm$^{-1}$')
ax_1.set_xticks(temp_2)
ax_1.set(xlabel='Temperature', ylabel='OD')
ax_1.plot(temp_2, peak_2, 'x')

In [ ]:
peak_3 = [peak_sub_128[0][i][-2] for i in range(len(peak_sub_128[0])) if i != 0]
sub_frequencies = sf.peak_freq(sub_peaks,shifted_sets, lims=[605,620])
freq_3 = [sub_frequencies[0][i][-2] for i in range(len(sub_frequencies[0])) if i != 0]
wavenumber_3 = [sf.converter(freq, d_type=1, c_type=0) for freq in freq_3]
print(wavenumber_3)
temp_3 = [10, 20, 30, 40, 140, 160, 180, 200, 220, 240, 260, 280]
peak_3.pop(0)
peak_3.pop(4)
peak_3.pop(5)
peak_3.pop(6)
peak_3.pop(7)
peak_3.insert(0, peak_sub_128[0][1][-3])
fig_2, ax_2 = mp.subplots()
ax_2.set_title('$\sigma$ polarisation at 16378.12 cm$^{-1}$')
ax_2.set(xlabel='Temperature', ylabel='OD')
ax_2.set_xticks(temp_3)
ax_2.plot(temp_3, peak_3, 'x')

In [ ]:
peak_4 = [peak_sub_128[0][i][-1] for i in range(len(peak_sub_128[0])) if i != 0]
sub_frequencies = sf.peak_freq(sub_peaks, shifted_sets, lims=[605,620])
freq_4 = [sub_frequencies[0][i][-1] for i in range(len(sub_frequencies[0])) if i != 0]
wavenumber_4 = [sf.converter(freq, d_type=1, c_type=0) for freq in freq_4]
print(wavenumber_4)
temp_4 = [10, 20, 30, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280]
fig_3, ax_3 = mp.subplots()
ax_3.set_title('$\sigma$ polarisation at 16174.57 cm$^{-1}$')
ax_3.set(xlabel='Temperature', ylabel='OD')
ax_3.set_xticks(temp_4)
ax_3.plot(temp_4, peak_4, 'x')

In [ ]:
peak173_0 = [value for values in peak_cor_173[3] for value in values]
od_frequencies173 = sf.peak_freq(corrected_peaks, shifted_sets, lims=[590,598])
freq173_0 = [frequency for frequencies in od_frequencies173[3] for frequency in frequencies]
wavenumber173_0 = [sf.converter(freq, 1, 0) for freq in freq173_0]
print(wavenumber173_0)

temp173_0 = [3.2, 10, 20, 30, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240]
fig_0, ax_0 = mp.subplots()
ax_0.set_title('$\pi$ polarisation at 16812.82 - 16773.16 cm$^{-1}$')
ax_0.set(xlabel='Temperature', ylabel='OD')
ax_0.plot(temp173_0, peak173_0, 'xr')

In [ ]:
peak173_2 = [peak_sub_173[3][i][0] for i in range(len(peak_sub_173[0])) if i != 0]
sub_frequencies = sf.peak_freq(sub_peaks, shifted_sets, lims=[597,610])
freq173_2 = [sub_frequencies[3][i][0] for i in range(len(sub_frequencies[3])) if i != 0]
wavenumber173_2 = [sf.converter(freq, d_type=1, c_type=0) for freq in freq173_2]
print(wavenumber173_2)
peak173_2.pop(0)
peak173_2.pop(1)
temp173_2 = [30, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280]

fig_4, ax_4 = mp.subplots()
ax_4.set_title('$\pi$ polarisation at 16704.25 cm$^{-1}$')
ax_4.set(xlabel='Temperature', ylabel='OD')
ax_4.set_xticks(temp173_2)
ax_4.plot(temp173_2, peak173_2, 'xr')


In [ ]:
peak173_3 = [peak_sub_173[3][i][1] for i in range(len(peak_sub_173[0])) if i != 0]
sub_frequencies = sf.peak_freq(sub_peaks, shifted_sets, lims=[597,610])
freq173_3 = [sub_frequencies[3][i][1] for i in range(len(sub_frequencies[3])) if i != 0]
wavenumber173_3 = [sf.converter(freq, d_type=1, c_type=0) for freq in freq173_3]
print(wavenumber173_3)
peak173_3.pop(0)
peak173_3.pop(1)
peak173_3.pop(2)
peak173_3.pop(3)
temp173_3 = [60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280]

fig_5, ax_5 = mp.subplots()
ax_5.set_title('$\pi$ polarisation at 16606.87 cm$^{-1}$')
ax_5.set(xlabel='Temperature', ylabel='OD')
ax_5.set_xticks(temp173_3)
ax_5.plot(temp173_3, peak173_3, 'xr')

In [ ]:
peak173_4 = [peak_sub_173[3][i][-1] for i in range(len(peak_sub_173[0])) if i != 0]
sub_frequencies = sf.peak_freq(sub_peaks, shifted_sets, lims=[597,610])
freq173_4 = [sub_frequencies[3][i][-1] for i in range(len(sub_frequencies[0])) if i != 0]
wavenumber173_4 = [sf.converter(freq, d_type=1, c_type=0) for freq in freq173_4]
print(wavenumber173_3)
peak173_4.pop(0)

temp173_4 = [20, 30, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280]

fig_6, ax_6 = mp.subplots()
ax_6.set_title('$\pi$ polarisation at 16444.17 cm$^{-1}$')
ax_6.set(xlabel='Temperature', ylabel='OD')
ax_6.set_xticks(temp173_4)
ax_6.plot(temp173_4, peak173_4, 'xr')

3P0 

In [ ]:
import matplotlib.pyplot as mp
temp = [3.2, 10, 20, 30, 40, 50, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280]
count = -1

fig_1, ax_1 = mp.subplots()
fig_2, ax_2 = mp.subplots()
fig_3, ax_3 = mp.subplots()

ax_1.set(xlabel='Temperature', ylabel='OD')
ax_2.set(xlabel='Temperature', ylabel='OD')
ax_3.set(xlabel='Temperature', ylabel='OD')

for index_1, OD_list in enumerate(peak_sub_128[0]):
    for index_2, value in enumerate(OD_list):
        if index_2 == 0:
            ax_1.plot(temp[index_1], value, 'xb')
        if index_1 > 9 and index_2 == 1:
            ax_2.plot(temp[index_1], value, 'xr')
        if index_2 == 2 or index_1 < 10 and index_2 == 2:
            ax_3.plot(temp[index_1], value, 'xg')


In [ ]:
import matplotlib.pyplot as mp
temp = [3.2, 10, 20, 30, 40, 50, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280]

fig_1, ax_1 = mp.subplots()
fig_2, ax_2 = mp.subplots()
fig_3, ax_3 = mp.subplots()

ax_1.set(xlabel='Temperature', ylabel='OD')
ax_2.set(xlabel='Temperature', ylabel='OD')
ax_3.set(xlabel='Temperature', ylabel='OD')

for index_1, OD_list in enumerate(peak_OD_173[3]):
    for index_2, value in enumerate(OD_list):
        if index_2 == 0:
            ax_1.plot(temp[index_1], value, 'xb')
        if index_2 == 1:
            ax_2.plot(temp[index_1], value, 'xr')
        if index_2 == 2:
            ax_3.plot(temp[index_1], value, 'xg')


In [ ]:
import fit_funcs as ff
import numpy as np
import matplotlib.pyplot as mp
from scipy.optimize import leastsq

x = np.linspace(0, 100, 100)
param_1 = [1, 5, 80, 8]
param_2 = [1, 0, 20, 5]
param_3 = [1, -2, 60, 5]
param_4 = [2, 0, 30, 10]

fit_param_1 = [1, 5, 70, 20]
fit_param_2 = [1, 0, 20, 20]
fit_param_3 = [1, -2, 60, 10]
fit_param_4 = [1, 0, 20, 20]
param = param_1 + param_2 + param_3 + param_4
fit_param = fit_param_1 + fit_param_2 + fit_param_3 + fit_param_4
y = ff.N_gaussian(x, *param)

fit, err = ff.fitNgauss(x, y, params=fit_param)

mp.plot(x, y)
mp.plot(x, ff.N_gaussian(x, *fit), 'x')
mp.show()